对citod数据处理
1. 删除长度为奇数的对话(实际是在data_load.py中处理的)
2. 去除标点, 以防bert的tokenizer分词会自动将标点识别为单个token(实际是在data_load.py中处理的)
3. 修改id,使得全局id不发生重复(实际是在gcn_data.py中处理,或者在pre.ipynb中处理的)
4. 删除长度为2的对话, 实际上在class dialogue_flow中处理的


In [12]:
import json
id_list = []
data_path = [f'./data/{x}/{x}_{y}.json' for x in ['navigate', 'calendar', 'weather'] for y in ['train', 'dev', 'test']]
for path in data_path:
    with open(path) as f:
        raw_data = json.load(f)
    for dialogue_components_item in raw_data:
        id_list.append(dialogue_components_item['id'])

for x in set(id_list):
    if id_list.count(x) > 1:
        print(x)
        break

82
126
246
278
346
405
423
558
582
802
853


In [5]:
import json
import re
# data_path = [f'./data/{x}/{x}_{y}.json' for x in ['navigate', 'calendar', 'weather'] for y in ['train', 'dev', 'test']] + [f'./data/{x}/entities.json' for x in ['navigate', 'calendar', 'weather']]
data_path = [f'./data/{x}/{x}_{y}.json' for x in ['navigate', 'calendar', 'weather'] for y in ['train', 'dev', 'test']] + ['./data/entities.json']
dia_id = 0
for path in data_path:
    with open(path, 'r+') as f:
        raw_data = f.read()
        # 0. 去除全文中的p_._f_._changs的点, 'p_. _f_._changs' in raw_data
        raw_data= re.sub(r'(p_\._f_\._changs|p_\.f_\._changs)', 'p_f_changs', raw_data)
        f.seek(0)
        f.truncate(0)
        f.write(raw_data)

In [6]:
import unicodedata
def _punctuation_string() -> str:
    """Get unicode punctuation character."""
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.

    all_punctuation = ''

    # Iterate over all Unicode characters
    for code_point in range(0x110000):
        char = chr(code_point)
        if (code_point >= 33 and code_point <= 47) or (code_point >= 58 and code_point <= 64) or (code_point >= 91 and code_point <= 96) or (code_point >= 123 and code_point <= 126):
            all_punctuation += chr(code_point)
            continue
        cat = unicodedata.category(char)
        if cat.startswith("P"):
            all_punctuation += chr(code_point)
    return all_punctuation

# punctuation_translator = str.maketrans('', '', _punctuation_string().replace('_','')) 不该删除标点, 而是标点用空格代替? 然后去除多余空格
punc_num = len(_punctuation_string().replace('_',''))
punctuation_translator = str.maketrans(_punctuation_string().replace('_',''), ' ' * punc_num)
new_tokenizer = lambda x: x.translate(punctuation_translator).split()

In [9]:
import json
data_path = [f'./data/{x}/{x}_{y}.json' for x in ['navigate', 'calendar', 'weather'] for y in ['train', 'dev', 'test']]
dia_id = 0
for path in data_path:
    with open(path, 'r+') as f:
        raw_data = json.load(f)
        new_raw_data = [] #还是别在源list上inplace操作,影响迭代次数本身
        for index, item in enumerate(raw_data):
            #1. 去除奇数长度
            if len(item['dialogue']) % 2: # 去除奇数数据,确保nn.mse loss不报错,否则长度为3的数据中q1,a1,q2中,sent_q=2,2,768但entity=2,1,768, shpae不同,mse报错
                continue # 不该continue, 而是要del掉, continue导致数据仍然存在,然后源数据上del导致影响for迭代次数
                del raw_data
            if len(item['dialogue']) == 1: # 去除不足两轮对话数据
                continue
            #2. 去除标点(换成空格,并去除多余空格)
            for i, utt in enumerate(item['dialogue']): # item["id"]没用到, 直接调用.values() # dia_dict[item["id"]].append(utt['utterance']) #item_id有重
                item['dialogue'][i]['utterance'] = " ".join(new_tokenizer(utt['utterance']))
            #3. 重建序号
            item['id'] = dia_id # 注意这里改变了原始listraw_data的内容
            dia_id += 1
            new_raw_data.append(item)
        f.seek(0)
        f.truncate(0)
        json.dump(new_raw_data, f, indent=4)

In [10]:
# 验证
import json
id_list = []
data_path = [f'./data/{x}/{x}_{y}.json' for x in ['navigate', 'calendar', 'weather'] for y in ['train', 'dev', 'test']]
for path in data_path:
    with open(path) as f:
        raw_data = json.load(f)
    for dialogue_components_item in raw_data:
        id_list.append(dialogue_components_item['id'])

for x in set(id_list):
    if id_list.count(x) > 1:
        print(x)

In [13]:
%cd ./data
import os
import json
import shutil

# Directories containing the files
navigate_dir = "navigate"
weather_dir = "weather"
calendar_dir = "calendar"

# Categories to process
categories = ["train.json", "dev.json", "test.json"]

# Output directory for the concatenated files
output_dir = "citod"
os.makedirs(output_dir, exist_ok=True)

for category in categories:
    navigate_file = os.path.join(navigate_dir, f"navigate_{category}")
    weather_file = os.path.join(weather_dir, f"weather_{category}")
    calendar_file = os.path.join(calendar_dir, f"calendar_{category}")

    with open(navigate_file, 'r') as navigate_f, open(weather_file, 'r') as weather_f, open(calendar_file, 'r') as calendar_f:
        navigate_data = json.load(navigate_f)
        weather_data = json.load(weather_f)
        calendar_data = json.load(calendar_f)

        concatenated_data=[]
        concatenated_data.extend(navigate_data)
        concatenated_data.extend(weather_data)
        concatenated_data.extend(calendar_data)

        output_file = os.path.join(output_dir, f"{output_dir}_{category}")
        with open(output_file, 'w') as output_f:
            json.dump(concatenated_data, output_f, indent=4)

for x in [navigate_dir, weather_dir, calendar_dir, output_dir]:
    shutil.copy("entities.json"), os.path.join(output_dir, "entities.json"))
%cd ../

/usr/local/zhoujiaming/kyoto/nlpcode/SemTransModel/SemTM/data
/usr/local/zhoujiaming/kyoto/nlpcode/SemTransModel/SemTM


In [2]:
x = """
    parser.add_argument('--dataset', type=str, default="navigate")
    parser.add_argument('--dia_enc', type=str, default="RNN_ParaGate_1")
    parser.add_argument('--sent_enc', type=str, default="UniAttention_1")
    parser.add_argument('--flow_type', type=str, default="FlowCellQA_E_2")
    parser.add_argument('--classifier', type=str, default="BiaffineClassifier")

    parser.add_argument('--save_path', type=str, default='./model.pt')
    parser.add_argument('--device', type=int, default=0)

    parser.add_argument('--hidden_dim', type=int, default=768)
    parser.add_argument('--dist_emb_size', type=int)
    parser.add_argument('--type_emb_size', type=int)
    parser.add_argument('--lstm_hid_size', type=int)
    parser.add_argument('--conv_hid_size', type=int)
    parser.add_argument('--bert_hid_size', type=int)
    parser.add_argument('--ffnn_hid_size', type=int)
    parser.add_argument('--biaffine_size', type=int)

    parser.add_argument('--dilation', type=str, help="e.g. 1,2,3") # default=[1,2,3]不能传list

    parser.add_argument('--emb_dropout', type=float)
    parser.add_argument('--conv_dropout', type=float)
    parser.add_argument('--out_dropout', type=float)

    # 训练相关
    parser.add_argument('--epochs', type=int, default=20)
    parser.add_argument('--batch_size', type=int, default=8)
    parser.add_argument('--seed', type=int, default=123)
    parser.add_argument('--device', type=int, default=1)
    parser.add_argument('--evaluate', action='store_true')
    parser.add_argument('--retoken', action='store_true')

    # 优化器权重衰减
    parser.add_argument('--clip_grad_norm', type=float, default=1.0)
    parser.add_argument('--learning_rate', type=float, default=5e-5)
    parser.add_argument('--weight_decay', type=float, default=1e-5)
    parser.add_argument('--bert_name', type=str)
    parser.add_argument('--bert_learning_rate', type=float, default=1e-5)
    parser.add_argument('--warm_factor', type=float)

    # other
    parser.add_argument('--use_bert_last_4_layers', type=int, help="1: true, 0: false")
    parser.add_argument('--noise_rate', type=float, default=0.00)
    parser.add_argument('--flow_bias', action='store_true')
"""
import re
for line in x.split('\n'):
    if match := re.search('--(.*?)\'', line):
        arg = match.group(1)
        print(f'{" "*8}self.{arg} = configs["{arg}"]')
    else: print(' '*8 + line.strip())

        
        self.dataset = configs["dataset"]
        self.dia_enc = configs["dia_enc"]
        self.sent_enc = configs["sent_enc"]
        self.flow_type = configs["flow_type"]
        self.classifier = configs["classifier"]
        
        self.save_path = configs["save_path"]
        self.device = configs["device"]
        
        self.hidden_dim = configs["hidden_dim"]
        self.dist_emb_size = configs["dist_emb_size"]
        self.type_emb_size = configs["type_emb_size"]
        self.lstm_hid_size = configs["lstm_hid_size"]
        self.conv_hid_size = configs["conv_hid_size"]
        self.bert_hid_size = configs["bert_hid_size"]
        self.ffnn_hid_size = configs["ffnn_hid_size"]
        self.biaffine_size = configs["biaffine_size"]
        
        self.dilation = configs["dilation"]
        
        self.emb_dropout = configs["emb_dropout"]
        self.conv_dropout = configs["conv_dropout"]
        self.out_dropout = configs["out_dropout"]
        
        # 训练相关
 

In [14]:
print('{')
for line in x.split('\n'):
    if match := re.search('--(.*?)\'', line):
        arg = match.group(1)
        if match := re.search(r'--(.*?)\'.*default=(.*)\)', line):
            val = match.group(2)
            print(f'{" "*2}"{arg}": {val},')
            continue
        print(f'{" "*2}"{arg}": 0,')
    else: print(' '*2 + line.strip())
print('}')

  
  "dataset": "navigate",
  "dia_enc": "RNN_ParaGate_1",
  "sent_enc": "UniAttention_1",
  "flow_type": "FlowCellQA_E_2",
  "classifier": "BiaffineClassifier",
  
  "save_path": './model.pt',
  "device": 0,
  
  "hidden_dim": 768,
  "dist_emb_size": 0,
  "type_emb_size": 0,
  "lstm_hid_size": 0,
  "conv_hid_size": 0,
  "bert_hid_size": 0,
  "ffnn_hid_size": 0,
  "biaffine_size": 0,
  
  "dilation": 0,
  
  "emb_dropout": 0,
  "conv_dropout": 0,
  "out_dropout": 0,
  
  # 训练相关
  "epochs": 20,
  "batch_size": 8,
  "seed": 123,
  "device": 1,
  "evaluate": 0,
  "retoken": 0,
  
  # 优化器权重衰减
  "clip_grad_norm": 1.0,
  "learning_rate": 5e-5,
  "weight_decay": 1e-5,
  "bert_name": 0,
  "bert_learning_rate": 1e-5,
  "warm_factor": 0,
  
  # other
  "use_bert_last_4_layers": 0,
  "noise_rate": 0.00,
  "flow_bias": 0,
  


In [2]:
y = """{
  "dataset": "navigate",
  "bert_name": "bert_base_cased",
  "dia_enc": "MultiUniAtten",
  "sent_enc": "MultiUniAtten",
  "flow_type": "FlowCellQA",
  "classifier": "BiaffineClassifier",

  "save_path": './model.pt',

  # 模型形状
  "hidden_size": 768,
  "max_position_embeddings": 2000,
  "type_emb_size": 2,
  "ffnn_hid_size": 3072,
  "head_count": 12,
  "num_hidden_layers": 12,

  "dilation": [1,2,3],

  "attn_dropout": 0.1,
  "emb_dropout": 0.5,
  "conv_dropout": 0.5,
  # "biaffine_dropout": 0.33,

  # 训练相关
  "epochs": 20,
  "batch_size": 8,
  "seed": 123,
  "device": 1,
  "evaluate": False,
  "retoken": False,

  # 优化器权重衰减
  "learning_rate": 1e-3,
  "weight_decay": 0,
  "clip_grad_norm": 1.0,

  "bert_learning_rate": 1e-5,
  "warm_factor": 0.1,
  "layer_norm_eps": 1e-12

  # other
  "noise_rate": 0.0,
  "flow_bias": 0,
}"""

In [9]:
# "noise_rate": 0.0 -> self.emb_dropout = config["emb_dropout"]
import re
for line in y.split('\n'):
    if match := re.search(r'\"(.*?)\"', line):
        arg = match.group(1)
        print(f'{" "*8}self.{arg} = config["{arg}"]')

    else: print(' '*8 + line.strip())

        {
        self.dataset = config["dataset"]
        self.bert_name = config["bert_name"]
        self.dia_enc = config["dia_enc"]
        self.sent_enc = config["sent_enc"]
        self.flow_type = config["flow_type"]
        self.classifier = config["classifier"]
        
        self.save_path = config["save_path"]
        
        # 模型形状
        self.hidden_size = config["hidden_size"]
        self.max_position_embeddings = config["max_position_embeddings"]
        self.type_emb_size = config["type_emb_size"]
        self.ffnn_hid_size = config["ffnn_hid_size"]
        self.head_count = config["head_count"]
        self.num_hidden_layers = config["num_hidden_layers"]
        
        self.dilation = config["dilation"]
        
        self.attn_dropout = config["attn_dropout"]
        self.emb_dropout = config["emb_dropout"]
        self.conv_dropout = config["conv_dropout"]
        self.biaffine_dropout = config["biaffine_dropout"]
        
        # 训练相关
        self.epochs

In [18]:
pp = """{
  "dataset": "navigate",
  "bert_name": "bert_base_cased",
  "dia_enc": "MultiUniAtten",
  "sent_enc": "SentUniAtten",
  "flow_type": "FlowCellQA",
  "classifier": "BiaffineClassifier",

  "save_path": "./model.pt",

  "hidden_size": 768,
  "max_position_embeddings": 2000,
  "type_emb_size": 2,
  "cln_hid_size": 3072,
  "ffnn_hid_size": 3072,
  "head_count": 12,
  "num_hidden_layers": 12,
  "flow_size": 2,

  "dilation": [1,2,3],

  "attn_dropout": 0.1,
  "emb_dropout": 0.5,
  "conv_dropout": 0.5,
  "layer_norm_eps": 1e-6,

  "epochs": 20,
  "batch_size": 8,
  "seed": 123,
  "device": 0,
  "evaluate": False,
  "retoken": False,

  "bert_learning_rate": 1e-5,
  "learning_rate": 5e-5,
  "warm_factor": 0.1,

  "clip_grad_norm": 1.0,
  "weight_decay": 0,

  "noise_rate": 0.0,
  "flow_bias": 0
}
"""

In [40]:
import json
json.loads(dd)

{'epochs': 20,
 'batch_size': 8,
 'seed': 123,
 'device': 0,
 'evaluate': False,
 'retoken': False}

In [39]:
dd  = """{
  "epochs": 20,
  "batch_size": 8,
  "seed": 123,
  "device": 0,
  "evaluate": false,
  "retoken": false

}"""

In [14]:
import torch
import torch.nn as nn
import geotorch

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # One line suffices: Instantiate a linear layer with orthonormal columns
        self.linear = nn.Linear(64, 128)
        geotorch.orthogonal(self.linear, "weight")


    def forward(self, x):
        # self.linear is orthogonal and every 3x3 kernel in self.cnn is of rank 1

# Use the model as you would normally do. Everything just works
model = Model().cuda()

# Use your optimizer of choice. Any optimizer works out of the box with any parametrization
optim = torch.optim.Adam(model.parameters(), lr=lr)

IndentationError: expected an indented block (2067904596.py, line 25)

In [24]:
import geotorch
layer = nn.Linear(2, 2)
geotorch.orthogonal(layer, "weight")
torch.allclose(layer.weight.T @ layer.weight, torch.eye(2,2))


# layer = nn.Conv2d(20, 40, 3, 3)  # Make the kernels orthogonal
# geotorch.orthogonal(layer, "weight")
# torch.norm(layer.weight.transpose(-2, -1) @ layer.weight - torch.eye(3,3))

True

In [25]:
layer = nn.Linear(20, 20)
M = FixedRank(layer.weight.size(), rank=6)
geotorch.register_parametrization(layer, "weight", M)
layer.weight = M.sample()

NameError: name 'FixedRank' is not defined

In [34]:
geotorch.GLp(torch.zeros(3,3).size()).sample()

tensor([[ 0.1120,  0.7717,  0.8428],
        [ 0.0864, -0.1902, -0.4740],
        [ 0.1904, -0.8738, -0.6036]])

tensor(True)